In [1]:
from utils.email_utils import send_email


In [2]:
send_email(to="sebastian@skills.tech",subject="test",content="Template corre de testing")


'Email sent successfully'

In [3]:
from supabase import create_client
from decouple import config

supabase = create_client(config("SUPABASE_URL"), config("SUPABASE_KEY"))


In [4]:
usuarios_prueba = [
    {
        "user_email": "sebastian@skills.tech",
        "reminder_type": "Learning Progress Check-In",
        "status": "pending",
        "scheduled_date": "2023-01-01T10:00:00Z",
        "content": "Just a quick reminder to check your progress on this week's topic...",
        "sent_date": None
    },
    {
        "user_email": "zuriel@skills.tech",
        "reminder_type": "Encouragement and Motivation",
        "status": "pending",
        "scheduled_date": "2023-01-02T10:00:00Z",
        "content": "You're doing an amazing job! Remember, every question you ask...",
        "sent_date": None
    },
    {
        "user_email": "sebastian@skills.tech",
        "reminder_type": "Learning Progress Check-In",
        "status": "pending",
        "scheduled_date": "2024-01-03T10:00:00Z",
        "content": "Just a quick reminder to check your progress on this week's topic...",
        "sent_date": None
    },
    # ... más usuarios de prueba ...
]

for usuario in usuarios_prueba:
    response = supabase.table("reminders_tb").insert(usuario).execute()
    try:
        if response.error:
            print(f"Error al insertar: {response.error}")
        else:
            print(f"Usuario insertado: {usuario['user_email']}")
    except:
        print("No se pudo cargar los datos ficticios")
        

2024-01-07 23:12:12,334:INFO - HTTP Request: POST https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb "HTTP/1.1 201 Created"
2024-01-07 23:12:12,452:INFO - HTTP Request: POST https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb "HTTP/1.1 201 Created"


No se pudo cargar los datos ficticios
No se pudo cargar los datos ficticios


2024-01-07 23:12:12,692:INFO - HTTP Request: POST https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb "HTTP/1.1 201 Created"


No se pudo cargar los datos ficticios


In [27]:
from utils.email_utils import Learning_process
import datetime
import pytz  # Asegúrate de tener instalado pytz
current_date = datetime.datetime.now(datetime.timezone.utc)


In [22]:
recordatorios = supabase.table("reminders_tb").select("*").eq("reminder_type", "Learning Progress Check-In").eq("status", "pending").execute().data

for recordatorio in recordatorios:
    print(recordatorio)
    #send_email(recordatorio["user_email"], "Learning Progress Check-In", Learning_process)

    # Actualizar el estado del recordatorio en la base de datos
    supabase.table("reminders_tb").update({"status": "sent"}).eq("id", recordatorio["id"]).execute()

2024-01-08 00:15:46,541:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&reminder_type=eq.Learning%20Progress%20Check-In&status=eq.pending "HTTP/1.1 200 OK"


Conexion con members y creacion de datos en supabase reminders_tb

In [29]:
# Obtener todos los usuarios de members_tb
members = supabase.table("members_tb").select("*").execute().data
for member in members:
    user_email = member["email"]
    user_name = member["name"]  # Obtener el nombre del usuario


    # Verificar si el usuario ya tiene un recordatorio en reminders_tb
    existing_reminder = supabase.table("reminders_tb").select("*").eq("user_email", user_email).eq("reminder_type", "Welcome").execute().data

    if not existing_reminder:
        # Enviar correo de bienvenida
        welcome_content = "Bienvenido a nuestra plataforma. ¡Estamos emocionados de tenerte aquí!"
        send_email(to=user_email, subject="Bienvenido", content=welcome_content)

        # Crear un registro de recordatorio para el nuevo usuario
        new_reminder = {
            "user_email": user_email,
            "name":user_name,
            "reminder_type": "Welcome",
            "status": "sent",
            "scheduled_date": None,  # No aplica para correos de bienvenida
            "content": "Bienvenido a nuestra plataforma.",
            "sent_date": current_date.isoformat()
        }
        supabase.table("reminders_tb").insert(new_reminder).execute()

        print(f"Correo de bienvenida enviado a {user_email} y recordatorio creado.")

2024-01-08 00:21:01,266:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/members_tb?select=%2A "HTTP/1.1 200 OK"
2024-01-08 00:21:01,365:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.lc%40luiszorrilla.com&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 00:21:01,471:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.test%40test.test&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 00:21:01,571:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.sebastian%40skills.tech&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 00:21:02,836:INFO - HTTP Request: POST https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb "HTTP/1.1 201 Created"
2024-01-08 00:21:02,937:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_

Correo de bienvenida enviado a sebastian@skills.tech y recordatorio creado.


2024-01-08 00:21:03,042:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.skills.demo%40newyorklife.com&reminder_type=eq.Welcome "HTTP/1.1 200 OK"


Envio de correos con html

In [8]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from decouple import config

email_user_cred=config("username")
email_user_pass=config("password")

def send_html_email(to, subject, html_content):
    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = "arturo@skills.tech"  # Cambiar a tu dirección de correo
    msg['To'] = to

    # Parte HTML del mensaje
    html_part = MIMEText(html_content, 'html')
    msg.attach(html_part)

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(email_user_cred, email_user_pass)  # Cambiar a tus credenciales
            server.send_message(msg)
        print(f"Correo enviado con éxito a {to}")
    except Exception as e:
        print(f"Error al enviar correo a {to}: {e}")


In [17]:
with open("email_templates/Learning_check_in.html", "r") as file:
        html_content = file.read()

html_content = html_content.replace("[Nombre del Usuario]", user_name)
send_html_email("sebastian@skills.tech", "Bienvenido a SkillsAI", html_content)

Correo enviado con éxito a sebastian@skills.tech


Correo de Learning check-in una semana despues

In [45]:
import datetime
import pytz  # Asegúrate de tener instalado pytz

current_date = datetime.datetime.now(datetime.timezone.utc)

users = supabase.table("members_tb").select("*").execute().data

for user in users:
    user_email = user["email"]

    # Buscar el recordatorio de bienvenida
    welcome_reminder = supabase.table("reminders_tb").select("*").eq("user_email", user_email).eq("reminder_type", "Welcome").execute().data

    if welcome_reminder:
        welcome_date = welcome_reminder[0]["sent_date"]
        if welcome_date:
            welcome_date = datetime.datetime.fromisoformat(welcome_date)
            if not welcome_date.tzinfo:
                welcome_date = pytz.utc.localize(welcome_date)

            scheduled_date = welcome_date + datetime.timedelta(days=7)

            # Buscar el recordatorio "Learning Progress Check-In"
            progress_checkin = supabase.table("reminders_tb").select("*").eq("user_email", user_email).eq("reminder_type", "Learning Progress Check-In").execute().data
            print(progress_checkin,current_date >= scheduled_date)
            if not progress_checkin:
                # Crear un registro para el nuevo recordatorio programado
                new_recordatorio = {
                    "user_email": user_email,
                    "user_name":user_name,
                    "reminder_type": "Learning Progress Check-In",
                    "status": "pending",
                    "scheduled_date": scheduled_date.isoformat(),
                    "content": "Learning Progress Check-In",
                    "sent_date": None
                }
                supabase.table("reminders_tb").insert(new_recordatorio).execute()
                print(f"Se programó email 'Learning Progress Check-In' para {user_email}")

            elif progress_checkin[0]["status"] == "pending" and current_date >= scheduled_date:
                # Enviar correo "Learning Progress Check-In"
                with open("email_templates/Learning_check_in.html", "r") as file:
                    html_content = file.read()

                    html_content = html_content.replace("[Nombre del Usuario]", user_name)
                    send_html_email(user_email, "Learning Progress Check-In", html_content)
                # Actualizar el estado del recordatorio
                supabase.table("reminders_tb").update({"status": "sent", "sent_date": current_date.isoformat()}).eq("id", progress_checkin[0]["id"]).execute()
                print(f"Se mando email 'Learning Progress Check-In' a {user_email}")



2024-01-08 01:01:07,220:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/members_tb?select=%2A "HTTP/1.1 200 OK"
2024-01-08 01:01:07,315:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.lc%40luiszorrilla.com&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 01:01:07,405:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.lc%40luiszorrilla.com&reminder_type=eq.Learning%20Progress%20Check-In "HTTP/1.1 200 OK"
2024-01-08 01:01:07,500:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.test%40test.test&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 01:01:07,597:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.test%40test.test&reminder_type=eq.Learning%20Progress%20Check-In "HTTP/1.1 200 OK"


[{'id': 57, 'user_email': 'lc@luiszorrilla.com', 'reminder_type': 'Learning Progress Check-In', 'status': 'sent', 'scheduled_date': '2024-01-15T02:31:17+00:00', 'content': 'Learning Progress Check-In', 'sent_date': None, 'user_name': None}] False
[{'id': 58, 'user_email': 'test@test.test', 'reminder_type': 'Learning Progress Check-In', 'status': 'sent', 'scheduled_date': '2024-01-15T06:31:13+00:00', 'content': 'Learning Progress Check-In', 'sent_date': None, 'user_name': None}] False


2024-01-08 01:01:07,690:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.sebastian%40skills.tech&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 01:01:07,783:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.sebastian%40skills.tech&reminder_type=eq.Learning%20Progress%20Check-In "HTTP/1.1 200 OK"
2024-01-08 01:01:07,877:INFO - HTTP Request: POST https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb "HTTP/1.1 201 Created"
2024-01-08 01:01:07,972:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.luis%40skills.tech&reminder_type=eq.Welcome "HTTP/1.1 200 OK"


[] False
Se programó email 'Learning Progress Check-In' para sebastian@skills.tech


2024-01-08 01:01:08,062:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.luis%40skills.tech&reminder_type=eq.Learning%20Progress%20Check-In "HTTP/1.1 200 OK"
2024-01-08 01:01:08,149:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.skills.demo%40newyorklife.com&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 01:01:08,240:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.skills.demo%40newyorklife.com&reminder_type=eq.Learning%20Progress%20Check-In "HTTP/1.1 200 OK"


[{'id': 59, 'user_email': 'luis@skills.tech', 'reminder_type': 'Learning Progress Check-In', 'status': 'sent', 'scheduled_date': '2024-01-15T06:31:01+00:00', 'content': 'Learning Progress Check-In', 'sent_date': None, 'user_name': None}] False
[{'id': 60, 'user_email': 'skills.demo@newyorklife.com', 'reminder_type': 'Learning Progress Check-In', 'status': 'sent', 'scheduled_date': '2024-01-15T06:31:09+00:00', 'content': 'Learning Progress Check-In', 'sent_date': None, 'user_name': None}] False


In [4]:
from database.supa import supabase_user,supabase_admin  # Importar los clientes de Supabase
from supabase import create_client
from decouple import config

In [5]:
url_admin: str = config("SUPABASE_ADMIN_URL")
key_admin: str = config("SUPABASE_ADMIN_KEY")

supabase_admin_2 = create_client(supabase_url=url_admin,supabase_key= key_admin)
admin_data = supabase_admin.table("courses_tb").select("*").eq("id", "ba86e360-577c-4145-baac-974611be0872").execute().data[0]
admin_data

2024-01-28 16:07:44,933:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/courses_tb?select=%2A&id=eq.ba86e360-577c-4145-baac-974611be0872 "HTTP/1.1 200 OK"


{'name': 'Fisica para todos ',
 'avatarid': '12b850ea-01c0-4b2f-9c68-e2ad9c655543',
 'companyid': '2e2aec92-0980-483e-9edd-05f187d10253',
 'syllabus': 'Fisica 1',
 'general_objective': 'Ensena fisica de manera clara y sencilla. ',
 'module_objective': 'Fisica de inicio a fin',
 'prerequisites': 'No hay',
 'reference_files': [{'0': '2e2aec92-0980-483e-9edd-05f187d10253/Fisica_para_todos_'}],
 'created_at': '2023-12-14T16:22:06.83592+00:00',
 'id': 'ba86e360-577c-4145-baac-974611be0872',
 'status': 'ready',
 'reference_videos': [],
 'categories': {'categories': [{'name': 'Classical Mechanics',
    'icon_url': 'https://example.com/icons/classical-mechanics.png',
    'description': 'Fundamentals of motion'},
   {'name': 'Thermodynamics',
    'icon_url': 'https://example.com/icons/thermodynamics.png',
    'description': 'Heat and energy studies'},
   {'name': 'Electromagnetism',
    'icon_url': 'https://example.com/icons/electromagnetism.png',
    'description': 'Electricity and magnetism'}

In [10]:
# Proyecto Usuario
url_user: str = config("SUPABASE_USER_URL")
key_user: str = config("SUPABASE_USER_KEY")
supabase_user = create_client(supabase_url=url_user,supabase_key= key_user)


response = supabase_user.table("responses_tb").select("*").eq("threadid", "4a37be7f-ce2c-4f19-aaaa-15f6d334a908").execute().data[0]
response

2024-01-28 16:10:04,286:INFO - HTTP Request: GET https://hhtrmdrpowaaxlydfwjn.supabase.co/rest/v1/responses_tb?select=%2A&threadid=eq.4a37be7f-ce2c-4f19-aaaa-15f6d334a908 "HTTP/1.1 200 OK"


{'id': 4,
 'threadid': '4a37be7f-ce2c-4f19-aaaa-15f6d334a908',
 'prompt': 'Cuales son las Leyes de Newton? ',
 'created_at': '2024-01-23T10:56:33.346471+00:00',
 'answer': 'Las tres leyes de Newton son fundamentales para entender la mecánica clásica. Aquí tienes una explicación conceptual de cada una de ellas, junto con un ejemplo práctico:\n\n1. **Primera Ley de Newton o Ley de la Inercia**: Esta ley establece que un objeto permanecerá en reposo o en movimiento rectilíneo uniforme a menos que actúe sobre él una fuerza externa. Conceptualmente, esta ley habla sobre la resistencia de los objetos a cambiar su estado de movimiento.\n    - **Ejemplo**: Cuando viajas en un automóvil y este frena repentinamente, tiendes a seguir moviéndote hacia adelante. Esto se debe a la inercia, tu cuerpo resiste el cambio en su estado de movimiento.\n2. **Segunda Ley de Newton**: Esta ley formula que la fuerza aplicada a un objeto es igual a la masa del objeto multiplicada por su aceleración (F = ma). En